##### Импортирую нужные библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'svg'")
%config InlineBackend.figure_format='svg'
# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.cmap'] = 'viridis'

##### Загружаю датасет 

In [2]:
df = pd.read_csv('RFM_ht_data.csv')

/nfs/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   InvoiceNo     332730 non-null  object 
 1   CustomerCode  332730 non-null  object 
 2   InvoiceDate   332730 non-null  object 
 3   Amount        332730 non-null  float64
dtypes: float64(1), object(3)
memory usage: 10.2+ MB


In [5]:
# меняю формат данных в датасете
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerCode'] = df['CustomerCode'].apply(str)
df['InvoiceNo'] = df['InvoiceNo'].apply(str)

#### RFM-анализ

In [14]:
# вывожу дату последнего действия, залогированного в датасете. От этой даты будут отсчитываться дни с момента последней покупки у пользователей.
last_date = df['InvoiceDate'].max()

In [15]:
last_date

Timestamp('2020-09-30 00:00:00')

In [16]:
# создаю новый датасет с расчетом метрик для RFM-анализа
rfmTablee = df.groupby('CustomerCode', as_index = False).agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency
                                      'InvoiceNo': lambda x: len(x),      # Frequency
                                      'Amount': lambda x: x.sum()}) # Monetary

In [17]:
rfmTablee.head()

,CustomerCode,InvoiceDate,InvoiceNo,Amount
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88


In [18]:
# привожу колонку order к формату int, переименовываю колонки
rfmTablee['InvoiceDate'] = rfmTablee['InvoiceDate'].astype(int)
rfmTablee.rename(columns = {'InvoiceDate':'recency',
                           'InvoiceNo':'frequency',
                           'Amount':'monetary_value'}, inplace = True)

In [19]:
rfmSegmentation = rfmTablee

In [ ]:
# распределяю пользователей по каждой из метрик на равные группы для RFM-анализа. Для этого считаю квантили метрик

In [20]:
quantiles = rfmTablee.quantile(q=[0.25,0.5,0.75])

In [21]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [22]:
# функция для R класса, т.к его логига откличается от остальных - чем меньше значение Recency, тем лучше
def RClass(value,columns,quantiles):
    if value <= quantiles[columns][0.25]:
        return 1
    elif value <= quantiles[columns][0.50]:
        return 2
    elif value <= quantiles[columns][0.75]:
        return 3
    else:
        return 4
    
# функция для F и M класса, у них другая логика, чем выше значение тем лучше класс
def FMClass(value, columns, quantiles):
    if value <= quantiles[columns][0.25]:
        return 4
    elif value <= quantiles[columns][0.50]:
        return 3
    elif value <= quantiles[columns][0.75]:
        return 2
    else:
        return 1

In [23]:
# создаю отдельные колонки со значением сегментов, применяя к метриками соответствующие функции из пердыдущего шага
rfmSegmentation['R_Quartile']=rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))
rfmSegmentation['F_Quartile']=rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))
rfmSegmentation['M_Quartile']=rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))


In [24]:
# получаю финальный сегмент RFM

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str)\
                        + rfmSegmentation.F_Quartile.map(str)\
                        + rfmSegmentation.M_Quartile.map(str)


In [25]:
# итоговый датасет
rfmSegmentation.head()

,CustomerCode,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


## Вопрос 1:
Какое максимальное кол-во покупок было совершено одним пользователем?

In [26]:
rfmSegmentation.frequency.max()

204

Ответ: 204

## Вопрос 2:
Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

In [27]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


Ответ: верхняя граница по квантилю 0.25 - 765.

## Вопрос 3:
Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?

In [28]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


Ответ: значение нижней границы количества покупок в подсегменте F - 4.

## Вопрос 4:
Какая верхняя граница у количества покупок у пользователей с классом 2 в подсегменте R?

In [37]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


Ответ: 2

## Вопрос 5: 
Сколько пользователей попало в сегмент 111?

In [34]:
rfmSegmentation.query('RFMClass == "111"').shape[0]

9705

Ответ: 9705

## Вопрос 6:
Сколько пользователей попало в сегмент 311

In [35]:
rfmSegmentation.query('RFMClass == "311"').shape[0]

1609

Ответ: 1609

## Вопрос 7:
В каком RFM-сегменте самое большое кол-во пользователей?

In [41]:
rfmSegmentation.groupby('RFMClass').size().sort_values(ascending = False)

RFMClass
444    10624
111     9705
443     6729
344     6593
211     5847
       ...  
424       63
214       60
114       60
314       33
414        2
Length: 64, dtype: int64

In [ ]:
Ответ: сегмент 444

## Вопрос 8:
В каком RFM-сегменте самое маленькое кол-во пользователей?

In [42]:
rfmSegmentation.groupby('RFMClass').size().sort_values(ascending = True)

RFMClass
414        2
314       33
114       60
214       60
424       63
       ...  
211     5847
344     6593
443     6729
111     9705
444    10624
Length: 64, dtype: int64

Ответ: 414